# 2. Data Preparation
📒 `2.0-rc-data-preparation.ipynb`

**Objetivo:** Transformar os dados brutos em um formato para uso em análise exploratória e modelagem.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# Utils
from src._utils import detect_outliers_iqr, plot_outliers, handle_outliers, analyze_outliers, plot_distribution_and_boxplot

# Ignore warnings
warnings.filterwarnings('ignore')

# Configura o matplotlib para mostrar gráficos inline
%matplotlib inline

# Setup para mostrar todas as colunas do dataframe
pd.set_option('display.max_columns', None)

## 1. Limpeza dos Dados


### a) Remove colunas desnecessárias e renomeia as colunas e classes

In [ ]:
# Carrega dados
df = pd.read_csv("../data/raw/bootcamp_train.csv")
print( "N° de linhas e colunas: ", df.shape)
df.head()

In [ ]:
# Deleta as colunas que não são necessárias
df.drop(columns=['id', 'id_produto'], inplace=True)

# Renomeia as colunas
cols_to_rename = {
    'tipo': 'tipo',
    'temperatura_ar': 'temperatura_ar',
    'temperatura_processo': 'temperatura_processo',
    'umidade_relativa': 'umidade_relativa',
    'velocidade_rotacional': 'velocidade_rotacional',
    'torque': 'torque',
    'desgaste_da_ferramenta': 'desgaste_da_ferramenta',
    'falha_maquina': 'falha_maquina',
    'FTE (Falha Tensao Excessiva)': 'Tensao Excessiva(FTE)',
    'FDC (Falha Dissipacao Calor)': 'Dissipacao de Calor(FDC)',
    'FP (Falha Potencia)': 'Falha de Potencia(FP)',
    'FDF (Falha Desgaste Ferramenta)': 'Desgaste da Ferramenta(FDF)',
    'FA (Falha Aleatoria)': 'Falha Aleatoria(FA)'
}

# Renomeia as colunas
df.rename(columns=cols_to_rename, inplace=True)


### Renomeia as classes da coluna `tipo` para facilitar a interpretação
dict_type = {
    'L': 'Baixa', 
    'M': 'Media', 
    'H': 'Alta'
    }
# Aplica o mapeamento
df['tipo'] = df['tipo'].map(dict_type)


# Visualiza as 5 primeiras linhas
print( "N° de linhas e colunas: ", df.shape)
print("Visualiza as 5 primeiras linhas:")
df.head()



### b) Valores Nulos

In [ ]:
# Verifica dados Nulos
df.isnull().sum()

In [ ]:
# Tratando Valores Nulos
# Preenchimento de valores ausentes com a mediana ( menos sensivel aos outliers)

num_cols_to_inspect = [
    'temperatura_ar', 
    'temperatura_processo', 
    'umidade_relativa', 
    'velocidade_rotacional', 
    'torque', 
    'desgaste_da_ferramenta'
    ]

for col in num_cols_to_inspect:
    df[col].fillna(df[col].median(), inplace=True)

print("Valores Nulos tratados com a mediana: \n", df.isnull().sum())


### c) Inconsistência nos dados

In [ ]:
# Verifica dados Inconsistentes
print("Descrição dos dados numéricos: \n", df.describe())
print("-"*50)

for col in num_cols_to_inspect:
    print(df.loc[df[col] < 0, col].value_counts())
    print("\n")

# Trata Inconsistências com a mediana
for col in num_cols_to_inspect:
    df.loc[df[col] < 0, col] = df[col].median()

print("Inconsistências tratadas!")

In [ ]:
# Verifica dados de classes com inconsistências
cols_to_inspect = [
    'falha_maquina', 
    'Desgaste da Ferramenta(FDF)', 
    'Dissipacao de Calor(FDC)', 
    'Falha de Potencia(FP)', 
    'Tensao Excessiva(FTE)', 
    'Falha Aleatoria(FA)'
    ]

# Verifica valores únicos em todas as colunas
for col in cols_to_inspect:
    print(f"Coluna: {col}")
    print(f"Valores únicos: {df[col].unique()}")
    print(f"Quantidade de valores únicos: {df[col].nunique()}")
    print("-"*50)


In [ ]:
# Dicionário de mapeamento para padronizar valores binários
map_classes = {
    # Valores que representam "0" (não/falso)
    'não': '0',
    'Não': '0',
    'N': '0',
    'False': '0',
    'false': '0',
    'nao': '0',
    '-': '0',
    # Valores que representam "1" (sim/verdadeiro)
    'sim': '1',
    'Sim': '1',
    'y': '1',
    'True': '1',
    'true': '1'
}

# Lista de colunas para aplicar a transformação
cols_to_transform = [
    'falha_maquina',
    'Desgaste da Ferramenta(FDF)',
    'Dissipacao de Calor(FDC)',
    'Falha de Potencia(FP)',
    'Tensao Excessiva(FTE)',
    'Falha Aleatoria(FA)'
]

# Aplica a transformação em cada coluna
for col in cols_to_transform:
    df[col] = df[col].astype(str)
    # Aplica o mapeamento
    df[col] = df[col].map(lambda x: map_classes.get(x, x))
    # Converte a coluna para tipo numérico
    df[col] = pd.to_numeric(df[col])

# Verifica os valores únicos após a transformação
print("Verificação após a transformação:")
print("-" * 50)
for col in cols_to_transform:
    print(f"Coluna: {col}")
    print(f"Valores únicos: {df[col].unique()}")
    print(f"Quantidade: {df[col].nunique()}")
    print("-" * 50)



In [ ]:
# Dicionário de mapeamento para o tipo de falha com os novos nomes das colunas
dict_tipo_falha = {
    'Desgaste da Ferramenta(FDF)': 'Desgaste da Ferramenta(FDF)', 
    'Dissipacao de Calor(FDC)': 'Dissipacao de Calor(FDC)', 
    'Falha de Potencia(FP)': 'Falha de Potencia(FP)', 
    'Tensao Excessiva(FTE)': 'Tensao Excessiva(FTE)', 
    'Falha Aleatoria(FA)': 'Falha Aleatoria(FA)'
}

# Função para determinar o tipo de falha usando os novos nomes
def get_tipo_falha(row):
    # Verifica cada coluna de falha
    for col in dict_tipo_falha.keys():
        try:
            if row[col] == 1:
                return dict_tipo_falha[col]
        except KeyError:
            continue
    return "Sem falhas"

# Criar a nova coluna tipo_falha
df['tipo_falha'] = df.apply(get_tipo_falha, axis=1)

# Verificar a distribuição dos tipos de falha
print("\nDistribuição dos tipos de falha:")
print(df['tipo_falha'].value_counts())


# Verificando se há inconsistências na classificação
#df[(df['falha_maquina'] == 1) & (df['tipo_falha'] == 'Sem falhas')]


### d) Dados Duplicados

In [ ]:
print("Dados Duplicados removidos: ", df.duplicated().sum())
df.drop_duplicates(inplace=True)

### e) Análise de Outliers

In [ ]:
# Análise da distribuição das variáveis numéricas
print("Análise da distribuição das variáveis numéricas:")

# Declarando as colunas alvo
target_cols = ['falha_maquina', 'Falha Aleatoria(FA)', 'Falha de Potencia(FP)', 'Tensao Excessiva(FTE)', 'Dissipacao de Calor(FDC)', 'Desgaste da Ferramenta(FDF)']

# Seleciona as colunas numéricas
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.drop(target_cols).tolist()

# Plotar a distribuição e o boxplot das variáveis numéricas
plot_distribution_and_boxplot(df_clean, numeric_cols)

# Mostrar estatísticas descritivas
print("\nEstatísticas descritivas das variáveis numéricas:")
print(df[numeric_cols].describe())

# Salvar o gráfico com melhor qualidade
plt.savefig('../reports/figures/2_data_preparation_outliers', 
            bbox_inches='tight', 
            dpi=300)


In [ ]:
# máscara acumulada de "é outlier em QUALQUER coluna"
any_outlier = pd.Series(False, index=df.index)

for col in numeric_cols:
    mask, stats = detect_outliers_iqr(df[col], col)
    any_outlier |= mask  # acumula outliers

# remover linhas que são outlier em qualquer coluna
df_clean = df.loc[~any_outlier].copy()

print(f"Linhas originais: {len(df)}")
print(f"Linhas removidas (outliers): {any_outlier.sum()}")
print(f"Linhas após limpeza: {len(df_clean)}")

# Análise resumida
outlier_analysis = analyze_outliers(df_clean, numeric_cols) 
print(outlier_analysis.to_string(index=False))


#### Remoção da variável de umidade relativa

NOta: A variável `umidade_relativa` não possui variação, possui pouca informação sobre os dados de falha, sendo assim, **esta variável será removida dos estudos.**

In [ ]:
df_clean.drop(columns=['umidade_relativa'], inplace=True)

df_clean.info()

#### Reordena as colunas e reseta index

In [ ]:
# Reset Index
df_clean.reset_index(drop=True, inplace=True)

# Reordena as colunas
cols_to_reorder = [
    'tipo', 
    'temperatura_ar', 
    'temperatura_processo', 
    'velocidade_rotacional', 
    'torque', 
    'desgaste_da_ferramenta', 
    'falha_maquina', 
    'tipo_falha',
    'Desgaste da Ferramenta(FDF)',
    'Dissipacao de Calor(FDC)',
    'Falha de Potencia(FP)',  
    'Tensao Excessiva(FTE)',
    'Falha Aleatoria(FA)',
    ]

# Reordena as colunas
df_clean = df_clean[cols_to_reorder]

# Salva o dataset tratado
path_to_save = "../data/processed/data_cleaned.csv"
df_clean.to_csv(path_to_save, index=False)

### Visualiza os Dados com Pandas Profile Report

In [ ]:
# Visualiza os dados pós tratamento com Pandas Profiling
from ydata_profiling import ProfileReport

FILE_PATH = "../reports/profile-reports/data_cleaned.html"

profile = ProfileReport(
    df_clean,
    title="Profiling Report - Dados Tratados"
)

# Salva HTML estático
profile.to_file(FILE_PATH)

## 2. Pré Processamento

In [258]:
# === Imports ===
import os
import numpy as np
import pandas as pd

# sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from joblib import dump

# Configuração
RANDOM_SEED = 42

In [259]:
# Carrega o dataset tratado
df = pd.read_csv('../data/processed/data_cleaned.csv')
# Dataset com features engineering
#df = pd.read_csv('../data/processed/data_cleaned_features_engineering.csv')

# Visualiza as 5 primeiras linhas
df.head()

,tipo,temperatura_ar,temperatura_processo,velocidade_rotacional,torque,desgaste_da_ferramenta,falha_maquina,tipo_falha,Desgaste da Ferramenta(FDF),Dissipacao de Calor(FDC),Falha de Potencia(FP),Tensao Excessiva(FTE),Falha Aleatoria(FA)
0,Baixa,298.3,309.1,1616.0,31.1,195.0,0,Sem falhas,0,0,0,0,0
1,Baixa,298.2,308.4,1388.0,53.8,137.0,0,Sem falhas,0,0,0,0,0
2,Baixa,298.2,307.8,1528.0,31.1,94.0,0,Sem falhas,0,0,0,0,0
3,Media,300.9,310.8,1599.0,33.0,7.0,0,Sem falhas,0,0,0,0,0
4,Baixa,299.6,310.5,1571.0,33.9,94.0,0,Sem falhas,0,0,0,0,0


In [260]:

# Carrega o dataset tratado
df = pd.read_csv('../data/processed/data_cleaned.csv')

# Variáveis Alvo (Y) e colunas que NÃO entram em X
# (mantemos a lista para evitar vazamento de informação)
target_vars = [
    'tipo_falha',                      # <- alvo agora é multiclasse
    'falha_maquina', 
    'Falha de Potencia(FP)', 
    'Dissipacao de Calor(FDC)', 
    'Tensao Excessiva(FTE)', 
    'Desgaste da Ferramenta(FDF)', 
    'Falha Aleatoria(FA)'
]

# Seleciona os atributos (X) e a variável alvo (y)
X = df.drop(columns=target_vars).copy()
y = df['tipo_falha'].copy()  # <- alvo multiclasse

# Identifica e seleciona os atributos numéricos e categóricos
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categoric_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

# Divide os dados em treino e teste (estratificado por classe)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_SEED,
    stratify=y
)

# Pré-processamento em Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categoric_cols)  # ignora categorias novas no teste
    ],
    remainder='drop'  # remove colunas não transformadas
)

print(f"Variável Alvo (y): {y.name}")
print(f"\nAtributos Numéricos (X): {numeric_cols}")
print(f"\nAtributos Categóricas (X): {categoric_cols}")

# Diagnóstico opcional: distribuição das classes (útil para estratégias futuras de balanceamento/ponderação)
print("\nDistribuição das classes (conjunto completo):")
print(y.value_counts(normalize=True).rename('proporcao').round(4))

Variável Alvo (y): tipo_falha

Atributos Numéricos (X): ['temperatura_ar', 'temperatura_processo', 'velocidade_rotacional', 'torque', 'desgaste_da_ferramenta']

Atributos Categóricas (X): ['tipo']

Distribuição das classes (conjunto completo):
tipo_falha
Sem falhas                     0.9848
Dissipacao de Calor(FDC)       0.0064
Tensao Excessiva(FTE)          0.0042
Falha Aleatoria(FA)            0.0020
Desgaste da Ferramenta(FDF)    0.0018
Falha de Potencia(FP)          0.0009
Name: proporcao, dtype: float64


### Salva os artefatos e dados tratados 

In [ ]:
# Persistência dos dados tratados
os.makedirs('../models/artifacts', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../data/interim', exist_ok=True)

# Salvar os splits para manter os dados tratados
X_train.to_csv('../data/interim/X_train.csv', index=False)
X_test.to_csv('../data/interim/X_test.csv', index=False)
y_train.to_csv('../data/interim/y_train.csv', index=False)
y_test.to_csv('../data/interim/y_test.csv', index=False)

# Salvar “especificação” do pré-processamento (listas de colunas + objeto definido/sem ajuste)
dump({
    "numeric_cols": numeric_cols,
    "categorical_cols": categoric_cols,
    "preprocessor": preprocessor,
    "random_seed": RANDOM_SEED,
    "target_name": "tipo_falha",
    "target_vars_all": target_vars,
    # utilidades para referência futura
    "classes_": sorted(y.unique().tolist()),
    "class_distribution_full": y.value_counts(normalize=True).to_dict()
}, "../models/artifacts/preprocessing_spec.joblib")

print("\nArtefatos salvos em ../models/artifacts e splits em ../data/interim.")



Artefatos salvos em ../models/artifacts e splits em ../data/interim.
